In [1]:
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

import spacy
import pickle
import csv
import re
from stop_words import get_stop_words

In [2]:
# df = pd.read_pickle('_outputs/harassement_clean_french_no_bots.pickle')
df = pd.read_pickle('_outputs/harassement_clean_french_no_bots_transports.pickle')

In [3]:
len(df)

2502

In [4]:
# noisy_pos_tags = ["PROP","DET","PART","CCONJ","ADP","PRON","VERB","ADJ"]
noisy_pos_tags = ["PROP","DET","PART","CCONJ","ADP","PRON"]
MIN_TOKEN_LENGTH = 3 # minimum token length to remove
MIN_WORDS_NUMBER = 10 # minimum of words

# Importe le corpus Français
import fr_core_news_sm
nlp = fr_core_news_sm.load()

In [5]:
def is_noise(token):
    '''
    standard way to validate spacy tokens
    This method validate all the passed tokens and set true false on it
    '''
    is_noise = False
    if token.pos_ in noisy_pos_tags:
        is_noise = True
        
    elif token.is_stop == True:
        is_noise = True
        
    elif token.is_digit == True:
        is_noise = True
        
    elif token.is_punct == True:
        is_noise = True
        
    elif token.is_space == True:
        is_noise = True
        
    elif token.is_alpha == False:
        is_noise = True
        
    elif len(token.string) <= MIN_TOKEN_LENGTH:
        is_noise = True
        
    return is_noise

In [6]:
fr_stop_words = get_stop_words('fr')
for stop in fr_stop_words:
    nlp.vocab[stop].is_stop = True

In [7]:
# remarque : c'est déjà partiellement nettoyé dans 01_clean_text
# je garde pour la "portabilité" des fonctions
def remove_specific_stop(words):
    my_stop_words = ['balance', 'ton', 'porc', 'metoo', 'balancetonporc', 'via', 'hashtag', 'too', 'html', 'php', 'the', 'and', 'this', 'new', 'news']
    for bad in my_stop_words:
        words = words.replace(bad, ' ')
    return words

In [8]:
texts = []
index = 1
for index, row in df.iterrows():
    article = []
    if index%10000 == 0:
        print(index) # pour voir où cela en est
    index += 1
    words = row['clean_text']
    words = remove_specific_stop(words)
    if(len(words.split()) >= MIN_WORDS_NUMBER): # Only tweets with at least 10 words
        nlp_words = nlp(words)
        for word in nlp_words:
            if not is_noise(word):
                article.append(word.lemma_)
        texts.append(article)

In [9]:
# with open("_outputs/tokens.pickle", "wb") as fp:
with open("_outputs/tokens_transports.pickle", "wb") as fp:
    pickle.dump(texts, fp)

In [10]:
texts[:10]

[['remonter',
  'manif',
  'immense',
  'olala',
  'mettre',
  'vie',
  'remonter',
  'aller',
  'gare',
  'peyrou',
  'montage',
  'mur',
  'tribunal'],
 ['toucher',
  'métro',
  'rue',
  'suivre',
  'insulter',
  'salir',
  'corps',
  'coller',
  'abuser',
  'indicible',
  'continue',
  'harcèlement',
  'sifflet',
  'propos',
  'salace',
  'remarque',
  'tenue',
  'jamais'],
 ['violence',
  'femme',
  'payet',
  'transport',
  'plaît',
  'laissez',
  'victime',
  'seul',
  'ignorez',
  'agression',
  'isme',
  'quotidien'],
 ['suivre',
  'sortie',
  'métro',
  'saison',
  'épisod',
  'cru',
  'naïvement',
  'temps',
  'changer',
  'donne',
  'idée',
  'minijupe'],
 ['ballot',
  'homme',
  'agresse',
  'sexuellement',
  'policier',
  'charger',
  'agression',
  'sexuel',
  'transport',
  'masturber',
  'condamner',
  'mois',
  'prison',
  'sursis'],
 ['rine',
  'millet',
  'libertin',
  'droit',
  'majorité',
  'marre',
  'femme',
  'prendre',
  'métro',
  'rine',
  'deneuve',
  'part